In [4]:
import sys
# Add parent directory to path to import file_paths module
sys.path.insert(0, '..')
import file_paths
import helper_functions

import eelbrain
from scipy.io import loadmat

In [5]:
SUBJECTS = helper_functions.get_subjects()

for SUBJECT in SUBJECTS:

    # Make sure the destination directory exists
    dst_dir = file_paths.EEG_DIR / SUBJECT
    dst_dir.mkdir(exist_ok=True, parents=True)

    eeg_dst = dst_dir / f'{SUBJECT}_eeg.pickle'
    padded_eeg_dst = dst_dir / f'{SUBJECT}_padded_eeg.pickle'

    # Skip processing if files already exist
    if eeg_dst.exists() and padded_eeg_dst.exists():
        print(f"Files for {SUBJECT} already exist, skipping.")
        continue

    print("Saving ~", SUBJECT, "...")

    subject_data_file = helper_functions.get_subject_data_file(SUBJECT)
    mat = loadmat(subject_data_file, squeeze_me=True, struct_as_record=False)
    data = mat['data']

    # --- get channel names ---
    ch_names = data.dim.chan.eeg[0][:64].tolist()

    # --- collect all trials ---
    eeg_trials = []
    padded_eeg_trials = []

    for trial in data.eeg:  # 60 trials
        eeg = trial[:, :64]        # remove extra channels

        # Create time axis
        time_axis = eelbrain.UTS(0, 1 / data.fsample.eeg, len(eeg))  # fsample rate is 64 Hz, so time step is 1/64 seconds

        # Create Sensor axis
        sensor_axis = eelbrain.Categorial(values=ch_names[:64], name='sensor')

        # Save the eeg data as an NDVar with dimensions (time, sensor)
        eeg = eelbrain.NDVar(eeg, dims=(time_axis, sensor_axis), name='eeg')

        # Save the padded eeg data as an NDVar with dimensions (time, sensor)
        padded_eeg = eelbrain.pad(eeg, tstart=-0.100, tstop=eeg.time[-1] + 1, name='padded_eeg')


        eeg_trials.append(eeg)
        padded_eeg_trials.append(padded_eeg)

    # --- concatenate trials ---
    eeg_concat = eelbrain.concatenate(eeg_trials)
    padded_eeg_concat = eelbrain.concatenate(padded_eeg_trials)

    eelbrain.save.pickle(eeg_concat, eeg_dst)
    eelbrain.save.pickle(padded_eeg_concat, padded_eeg_dst)
    print(f"Saved eeg and padded eeg for {SUBJECT}")


Files for S1 already exist, skipping.
Files for S2 already exist, skipping.
Files for S3 already exist, skipping.
Files for S4 already exist, skipping.
Files for S5 already exist, skipping.
Files for S6 already exist, skipping.
Files for S7 already exist, skipping.
Files for S8 already exist, skipping.
Files for S9 already exist, skipping.
Files for S10 already exist, skipping.
Files for S11 already exist, skipping.
Files for S12 already exist, skipping.
Files for S13 already exist, skipping.
Files for S14 already exist, skipping.
Files for S15 already exist, skipping.
Files for S16 already exist, skipping.
Files for S17 already exist, skipping.
Files for S18 already exist, skipping.


In [6]:
# Sanity check: load one of the saved files and print its shape
eeg = eelbrain.load.unpickle(file_paths.EEG_DIR / "S1" / "S1_eeg.pickle")
padded_eeg = eelbrain.load.unpickle(file_paths.EEG_DIR / "S1" / "S1_padded_eeg.pickle")
print(eeg, padded_eeg)

<NDVar 'eeg': 192000 time, 64 sensor> <NDVar 'padded_eeg': 196200 time, 64 sensor>
